In [2]:
from typing import Tuple, List
import pandas as pd
import numpy as np
import seaborn as sns
import xgboost as xgb

import matplotlib.pyplot as plt
import plotly.graph_objects as go 
import plotly.express as px

from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import normalize, LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

from xgboost import XGBRegressor

In [ ]:
from google.colab import files

# Affiche un bouton pour télécharger des fichiers
uploaded = files.upload()


Data

In [10]:
hum = pd.read_csv('nuclear-waste/Training_data_humidity.csv')
pre = pd.read_csv('nuclear-waste/Training_data_pressure.csv')
ex = pd.read_csv('nuclear-waste/example_of_submission.csv')

df_tem = pd.read_csv('nuclear-waste/Training_data_temperature.csv')                   
df_sensors = pd.read_csv('nuclear-waste/Coordinates_Training.csv')


In [ ]:
# With the scatter_3d function from plotly, we will visualize the sensors position in space

fig = px.scatter_3d(
    df_sensors,
    x='Coor X [m]',
    y='Coor Y [m]',
    z='Coor Z [m]',
    width = 800,
    height = 600,
    hover_name='Sensor ID' 
)

fig.update_traces(marker=dict(size=3))

fig.show()


Cylinder shape, disc on XZ plane

In [ ]:
# Preprocessing

In [11]:
# Combining the data on one data frame will facilitate the access to them

df_long = df_tem.melt(id_vars='M.Time[d]', var_name='Sensor ID', value_name='Temperature')
df_merged = df_long.merge(df_sensors[['Sensor ID', 'Index', 'Material','Coor X [m]','Coor Y [m]','Coor Z [m]', 'R [m]']], on='Sensor ID', how='left')

# Label encoder, numeralize the material to make it a feature
#le = LabelEncoder()
#df_merged['Material_encoded'] = le.fit_transform(df_merged['Material'])

In [26]:
def eval(
    y_val : pd.DataFrame,
    y_pred : pd.DataFrame,
    model
):
    """
    Outputs the Mean Squared Error (MSE) and the coefficient of determination (R²)
    y_val : DataFrame of validation data
    y_pred : DataFrame of prediction data
    model : model
    """
    mse = mean_squared_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)

    print(f"MSE : {mse:.4f}")
    print(f"R² : {r2:.4f}")
    print(f"Features importances : {model.feature_importances_}")

In [13]:
# Replace with value

def replacewithvalue(
    df : pd.DataFrame,
    header : str,
    val = np.nan,
    min_val = -float('inf') ,
    max_val = float('inf') ,
) -> int:
    """
    Replace values between a certain interval with a unique value
    Clipping

    df : DataFrame
    header : Header of column
    value : Value used to replace, NaN by default
    min_val : minimal value of the interval
    max_val : maximal value of the interval
    """
    for i in df[header]:
        if min_val < i < max_val and (type(i) == float or type(i) == str) :
            df.replace(to_replace = i, value=val, inplace= True)
    return df



In [14]:
# Here we will replace NaNs with another value. For example, the arithmetric average

def avgcolumn(
    df : pd.DataFrame,
    header : str,
    min_val = -float('inf') ,
    max_val = float('inf') ,
) -> int:
    """
    Returns average of a DateFrame column

    df : The target DataFrame
    header : Header of column
    min_val : minimal value of the interval
    max_val : maximal value of the interval
    """
    liste = []
    for i in df[header]:
        if min_val < i < max_val and (type(i) == float or type(i) == str)  :
            liste.append(i)
    if len(liste) <=0 :
        return 0
    else :
        return sum(liste)/len(liste)


In [15]:
def correlate(df: pd.DataFrame,
              label1: str,
              label2: str
):
    correlation = df[label1].corr(df[label2])
    print(f"Corrélation pression-température : {correlation:.3f}")
    
    # Visualise avec un nuage de points
    sns.scatterplot(data=df, x=label1, y=label2)
    plt.title(f'Scatter Pression vs Température (corr = {correlation:.2f})')
    plt.xlabel(label1)
    plt.ylabel(label2)
    plt.grid(True)
    plt.show()


In [16]:
# Make it so it removes sensors

def removesensor(
    df: pd.DataFrame, 
    sensor: str
)-> pd.DataFrame:
    """
    Removes sensor and it's data from the DataFrame
    df : The target DataFrame
    sensor : The removed sensor
    """
    df_cleaned = df[df['Sensor ID'] != sensor]
    return df_cleaned

In [17]:
# Let's apply these functions

average = avgcolumn(df_merged,'Temperature', 0.0, 1000.0)
replacewithvalue(df_merged,'Temperature', min_val = 1000.0)
df_merged = df_merged.fillna(average)


In [18]:
#Adding humidity as a feature

df_hum = hum.melt(id_vars='M.Time[d]', var_name='Sensor ID', value_name='Humidity')
averageh = avgcolumn(df_hum,'Humidity')
df_hum = df_hum.fillna(averageh)
df_merged = df_merged.merge(df_hum[['Sensor ID', 'Humidity']], on='Sensor ID', how='left')


In [19]:
# Adding pressure as a feature

df_pre = pre.melt(id_vars='M.Time[d]', var_name='Sensor ID', value_name='Pressure')
averagep = avgcolumn(df_pre,'Pressure')
df_pre = df_pre.fillna(averagep)
df_merged = df_merged.merge(df_pre[['Sensor ID', 'M.Time[d]', 'Pressure']], on=['Sensor ID', 'M.Time[d]'], how='left')





In [20]:
# Remove N_442

df_merged = removesensor(df_merged,'N_442')



In [21]:
# Now we combine the sets and split them into subsets 

X = df_merged[['M.Time[d]','Coor X [m]','Coor Y [m]','Coor Z [m]','R [m]','Material','Humidity','Pressure']]
X = X.rename(columns={
    'M.Time[d]': 'Time',
    'Coor X [m]': 'X',
    'Coor Y [m]': 'Y',
    'Coor Z [m]': 'Z',
    'R [m]': 'R'
})
y = df_merged[['Temperature']]


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=10)

In [22]:
# Material encoder

material_target_mean = X_train.assign(temp=y_train).groupby('Material')['temp'].mean()
X_train['Material_encoded'] = X_train['Material'].map(material_target_mean)
X_val['Material_encoded'] = X_val['Material'].map(material_target_mean).fillna(y_train.mean())

X_train = X_train.drop('Material', axis=1)
X_val = X_val.drop('Material', axis=1)

In [ ]:
# We tested Linear regression L1, L2 and KNN Regressor --> Not good

In [ ]:
# Random Forest Regressor 

rf = RandomForestRegressor(n_estimators=50, max_depth = 20, random_state=5)
a = rf.fit(X_train, y_train.values.ravel())
y_predrf = rf.predict(X_val)


In [ ]:
eval(y_val, y_predrf)

In [23]:
# XGBoost

xgbm = XGBRegressor(n_estimators= 85, max_depth= 6, learning_rate= 0.09, random_state= 6)
xgbm.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], verbose=True)
y_predxgbm = xgbm.predict(X_val)


[0]	validation_0-rmse:14.08728	validation_1-rmse:13.95759
[1]	validation_0-rmse:12.94211	validation_1-rmse:12.82379
[2]	validation_0-rmse:11.90017	validation_1-rmse:11.79291
[3]	validation_0-rmse:10.96445	validation_1-rmse:10.86658
[4]	validation_0-rmse:10.09767	validation_1-rmse:10.00955
[5]	validation_0-rmse:9.31145	validation_1-rmse:9.23235
[6]	validation_0-rmse:8.58877	validation_1-rmse:8.51618
[7]	validation_0-rmse:7.94110	validation_1-rmse:7.87466
[8]	validation_0-rmse:7.35617	validation_1-rmse:7.29596
[9]	validation_0-rmse:6.82501	validation_1-rmse:6.76997
[10]	validation_0-rmse:6.34745	validation_1-rmse:6.29846
[11]	validation_0-rmse:5.91561	validation_1-rmse:5.87127
[12]	validation_0-rmse:5.52015	validation_1-rmse:5.47969
[13]	validation_0-rmse:5.17422	validation_1-rmse:5.13634
[14]	validation_0-rmse:4.86348	validation_1-rmse:4.82916
[15]	validation_0-rmse:4.57588	validation_1-rmse:4.54315
[16]	validation_0-rmse:4.31717	validation_1-rmse:4.28588
[17]	validation_0-rmse:4.08434	

In [24]:
eval(y_val, y_predxgbm, xgbm)
X_train

MSE : 2.9681
R² : 0.9872
[0.07939128 0.00258005 0.03845801 0.01140915 0.28075388 0.01290971
 0.03460988 0.539888  ]


,Time,X,Y,Z,R,Humidity,Pressure,Material_encoded
289257,1726,-15.537055,5.765402,10.251785,18.614488,100.000000,1471.379849,21.391313
278108,1895,-9.916048,20.887589,8.646642,13.156460,100.000000,1487.515994,21.391313
102717,1595,-0.520624,19.514188,-3.930985,3.965311,100.000000,813.464319,21.391313
873914,1662,-0.158756,27.409982,0.496578,0.521338,80.923442,-74705.813430,106.583700
875872,1621,-2.317018,15.831925,2.840820,3.665901,100.000000,809.625373,21.391313
...,...,...,...,...,...,...,...,...
618865,1621,-2.876773,23.805368,1.090982,3.076697,100.000000,395.515591,21.391313
443712,1606,5.455703,34.617074,0.872514,5.525032,100.000000,1018.649933,21.391313
882191,1770,-3.483447,34.895662,0.184400,3.488324,100.000000,785.494033,21.391313
761981,1560,1.858000,35.627803,-0.662457,1.972565,99.540328,-796.974578,21.391313


In [ ]:
# Gradient Boosting Regression

gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.08)
gbr.fit(X_train, y_train.values.ravel())
y_predgbr = gbr.predict(X_val)


In [ ]:
eval(y_val, y_predgbr)

In [ ]:
Cat

In [ ]:
# Comparisons

# Random Forest Regressor
#
#XGBoost
#
#Gradient Boosting Regression

In [ ]:
cv_scores = cross_val_score(rf, X_train, y_train.values.ravel(), cv=5, scoring='neg_mean_squared_error')
print("MSE CV RF :", -cv_scores.mean())

In [ ]:
cv_scores = cross_val_score(xgbm, X_train, y_train.values.ravel(), cv=5, scoring='neg_mean_squared_error')
print("MSE CV XGB :", -cv_scores.mean())

In [ ]:
cv_scores = cross_val_score(gbr, X_train, y_train.values.ravel(), cv=5, scoring='neg_mean_squared_error')
print("MSE CV GBR :", -cv_scores.mean())

In [ ]:
# Hyperparameters search

In [ ]:
# Définir la grille d'hyperparamètres à tester
param_grid = {
    'n_estimators': [75, 80, 85, 90],
    'max_depth': [6, 7, 8],
    'learning_rate': [0.08,0.085,0.09,0.095]
}

# Configurer la recherche par validation croisée
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    cv=5,  # nombre de folds pour la validation croisée
    scoring='neg_mean_squared_error',  # ou 'r2', 'neg_mean_absolute_error', etc.
    verbose=1,
    n_jobs=-1  # utilise tous les cœurs disponibles
)

# Lancer la recherche
grid_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres trouvés
print("Meilleurs hyperparamètres :", grid_search.best_params_)

# Utiliser le meilleur modèle pour prédire
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val)

In [ ]:
# Apply scaling to the columns, which is especially important for neural networks later
# Step 1: Define columns to scale (INPUTS only)
features_to_scale = [
    "M.Time[d]", "Coor X [m]", "Coor Y [m]", "Coor Z [m]", "R [m]",
    "Humidity", "Pressure"
]

# Step 2: Initialize scalers
input_scaler = StandardScaler()
target_scaler = StandardScaler()

# Step 3: Copy original DataFrame to avoid overwriting
df_merged_s = df_merged.copy()

# Step 4: Scale input features
df_merged_s[features_to_scale] = input_scaler.fit_transform(df_merged[features_to_scale])

# Step 5: Scale the target separately and store it as a new column
df_merged_s["Temperature_scaled"] = target_scaler.fit_transform(df_merged[["Temperature"]])
df_merged_s

In [ ]:
# Step 1: Define X using the scaled features (including humidity and pressure!)
X_s = df_merged_s[['M.Time[d]', 'Coor X [m]', 'Coor Y [m]', 'Coor Z [m]', 'R [m]',
                   'Humidity', 'Pressure', 'Material']]

# Step 2: Rename columns (optional, for readability)
X_s = X_s.rename(columns={
    'M.Time[d]': 'Time',
    'Coor X [m]': 'X',
    'Coor Y [m]': 'Y',
    'Coor Z [m]': 'Z',
    'R [m]': 'R'
})

# Step 3: Use the SCALED temperature as your target
y_s = df_merged_s[['Temperature_scaled']]

# Step 4: Split into training and validation sets
X_train_s, X_val_s, y_train_s, y_val_s = train_test_split(X_s, y_s, test_size=0.2, random_state=1)

X_train_s

In [ ]:
# Material encoder

material_target_mean = X_train_s.assign(temp=y_train_s).groupby('Material')['temp'].mean()
X_train_s['Material_encoded'] = X_train_s['Material'].map(material_target_mean)
X_val_s['Material_encoded'] = X_val_s['Material'].map(material_target_mean).fillna(y_train_s.mean())

X_train_s = X_train_s.drop('Material', axis=1)
X_val_s = X_val_s.drop('Material', axis=1)
X_train_s

In [ ]:
# Neural Network
# Initialize the neural network
nn = MLPRegressor(
    hidden_layer_sizes=(100, 50),  # you can experiment with these
    activation='relu',
    solver='adam',
    max_iter=300,
    random_state=1
)

# Fit the model
nn.fit(X_train_s, y_train_s.values.ravel())

# Predict (scaled)
y_pred_scaled = nn.predict(X_val_s)

# Inverse transform predictions and targets
y_pred = target_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1))
y_val_real = target_scaler.inverse_transform(y_val_s)

# Evaluate
mse = mean_squared_error(y_val_real, y_pred)
r2 = r2_score(y_val_real, y_pred)

print(f"MSE : {mse:.4f}")
print(f"R²  : {r2:.4f}")

In [ ]:
# Final Test 

In [ ]:
df_test = pd.read_csv('nuclear-waste/Coordinates_Test.csv')
humtest = pd.read_csv('nuclear-waste/Test_Time_humidity.csv')
pretest = pd.read_csv('nuclear-waste/Test_Time_pressure.csv')

In [ ]:
#le2 = LabelEncoder()
#df_test['Material_encoded'] = le2.fit_transform(df_test['Material'])
df_test["key"] = 1
df_times = pd.DataFrame({"M.Time[d]": df_tem['M.Time[d]']})
df_times["key"] = 1
df_test_prepared = pd.merge(df_test, df_times, on="key").drop("key", axis=1)

In [ ]:
df_humtest = humtest.melt(id_vars='M.Time[d]', var_name='Sensor ID', value_name='Humidity')
df_test_prepared = df_test_prepared.merge(df_humtest, on=['Sensor ID', 'M.Time[d]'], how='left')

In [ ]:
df_pretest = pretest.melt(id_vars='M.Time[d]', var_name='Sensor ID', value_name='Pressure')
df_test_prepared = df_test_prepared.merge(df_pretest, on=['Sensor ID', 'M.Time[d]'], how='left')

In [ ]:
df_test_prepared = df_test_prepared[['M.Time[d]','Sensor ID','Coor X [m]', 'Coor Y [m]', 'Coor Z [m]', 'R [m]', 'Material','Humidity','Pressure']]

df_test_prepared = df_test_prepared.rename(columns={
    'M.Time[d]': 'Time',  
    'Sensor ID': 'Sensor ID',
    'Coor X [m]': 'X',
    'Coor Y [m]': 'Y',
    'Coor Z [m]': 'Z',
    'R [m]': 'R'
})

X_test = df_test_prepared[['Time','X', 'Y', 'Z', 'R', 'Material', 'Humidity', 'Pressure']]


In [ ]:
# Material encoder
X_test['Material_encoded'] = X_test['Material'].map(material_target_mean).fillna(y_train.mean())

X_test = X_test.drop('Material', axis=1)

In [ ]:
# MODIFY MODEL 
         # HERE 
        #   v
y_pred_t = xgbm.predict(X_test)
y_pred_t.shape
y_pred_t = y_pred_t.reshape(int(4640/32),32)

In [ ]:
header = df_tem['M.Time[d]'].to_numpy()
header = header.astype(str)
header[header == '0'] = 'id'
header = list(header)

In [ ]:
ex['id']
ids = ex['id'].to_numpy()

In [ ]:
final = pd.DataFrame(y_pred_t, columns=header)
final.insert(0, "id", ids)


In [ ]:
final.to_csv("Results/final36.csv", index=False)

In [ ]:
# Initialiser le modèle
xgbm = XGBRegressor(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.09,
    objective='reg:squarederror',  # 'reg:absoluteerror' si tu veux MAE native
    random_state=6
)

# Stocker les prédictions à chaque round
train_preds = []
val_preds = []

xgbm.set_params(eval_metric="mae")
xgbm.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], verbose=True)


# Accéder aux prédictions intermédiaires
for i in range(1, len(xgbm.evals_result()['validation_0']['mae']) + 1):
    train_pred = xgbm.predict(X_train, iteration_range=(0, i))
    val_pred = xgbm.predict(X_val, iteration_range=(0, i))
    train_preds.append(train_pred)
    val_preds.append(val_pred)

# Calculer les métriques à chaque étape
train_mae = [mean_absolute_error(y_train, p) for p in train_preds]
val_mae   = [mean_absolute_error(y_val, p) for p in val_preds]

train_mse = [mean_squared_error(y_train, p) for p in train_preds]
val_mse   = [mean_squared_error(y_val, p) for p in val_preds]

train_r2  = [r2_score(y_train, p) for p in train_preds]
val_r2    = [r2_score(y_val, p) for p in val_preds]

# Tracer les courbes
plt.figure(figsize=(15, 12))

plt.subplot(3, 1, 1)
plt.plot(train_mae, label='Train MAE')
plt.plot(val_mae, label='Val MAE')
plt.ylabel('MAE')
plt.legend()
plt.grid(True)

plt.subplot(3, 1, 2)
plt.plot(train_mse, label='Train MSE')
plt.plot(val_mse, label='Val MSE')
plt.ylabel('MSE')
plt.legend()
plt.grid(True)

plt.subplot(3, 1, 3)
plt.plot(train_r2, label='Train R²')
plt.plot(val_r2, label='Val R²')
plt.xlabel('Boosting Rounds')
plt.ylabel('R²')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()
